In [2]:
import sys
import pickle
import numpy as np
import pandas as pd
#from nlp_utils import clean_text, pos_tag_text
sys.path.append("../")
from param_config import config
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
import xgboost as xgb

pd.set_option('display.float_format',lambda x: '%.5f'%x)

print("Load data...")

dfTrain = pd.read_csv(config.original_train_data_path)

dfPred= pd.read_csv(config.original_test_data_path)
dfPred2= pd.read_csv(config.original_test_data2_path)
# number of train/test samples
num_train, num_pred = dfTrain.shape[0], dfPred.shape[0]

print("Done.")

f = open('../../Data/col_name.txt','r')  
a = f.read()  
col_name = eval(a)  
f.close() 

f = open('../../Data/Procedure.txt','r')  
a = f.read()  
procedure = eval(a)  
f.close() 

dfTrain = dfTrain.rename(columns=col_name)
raw_predictors = dfTrain.columns.tolist()[1:-1]
dfPred = dfPred.rename(columns=col_name)
dfPred2 = dfPred2.rename(columns=col_name)

dfPredAll = pd.concat([dfPred,dfPred2],axis=0)

dfAll = pd.concat([dfTrain,dfPred,dfPred2],axis=0)

#dfAll = dfAll.replace({'':np.nan,0:np.nan})
dfAll = dfAll.reset_index()

Load data...


FileNotFoundError: File b'../Data/train_non_missing_v2.csv' does not exist

In [3]:
dfAll = pd.read_csv('../../Data/All_non_missing_v2.csv')

In [33]:
dfTrain_tmp = dfAll.merge(dfTrain[['ID','Y']],'inner','ID')
dfPred_tmp = dfAll.loc[dfAll['ID'].isin(dfPred['ID'])]
dfPred2_tmp = dfAll.loc[dfAll['ID'].isin(dfPred2['ID'])]

In [38]:
dfTrain_tmp.to_csv('../../Data/train_non_missing_v2.csv',index=False)
dfPred_tmp.to_csv('../../Data/test_a_non_missing_v2.csv',index=False)
dfPred2_tmp.to_csv('../../Data/test_b_non_missing_v2.csv',index=False)

In [ ]:
dfAll[pro_feat] = dfAll[pro_feat]-dfAll[pro_feat].mean()

In [ ]:
#dfAll = dfAll.replace({-99999999:-2.2222222})
dfAll[['750X1450','750X1451']]

In [ ]:
#dfAll = dfAll.replace({np.nan:-99999999})
pro_feat = [var for var in raw_predictors if not 'TOOL' in var ]
corr = dfAll[pro_feat].corr()
for i in range(len(corr)):
    corr.iloc[i,i:] =np.nan
print((corr==1.0).sum(axis=1))

In [ ]:
corr

In [ ]:
#处理0这个比较异常的值,在trian set中有40条数据，在某些column上为0 ，需要处理，test_a中也有一条
feature = ['ID']
for var in raw_predictors:
    if 'TOOL' in var:
        feature.append(var)
pick_list = (((corr==1.0).sum(axis=1))==0).values
for i in range(len(pick_list)):
    if pick_list[i]==True:
        feature.append(corr.index[i])
print(len(feature))

In [ ]:
#number of NAN
dfAll['Total_NAN_Num']=dfAll[raw_predictors].T.isnull().sum().tolist()
for key,pro in procedure.items():
    dfAll[key+'_NAN_Num']=dfAll[pro].T.isnull().sum().tolist()

In [5]:
raw_predictors = dfAll.columns.tolist()[1:-13]
raw_predictors[-1]

'750X1384'

In [ ]:
#Count 计算特定值在变量中重复出现的次数'''
dfAll = dfAll.replace({np.nan:-99999999})
added = ['ID']
for var in raw_predictors:
    if not var in dfAll.columns:
        continue
    dfAll['cnt_'+var] = 0
    added.append('cnt_'+var)
    groups = dfAll.groupby([var])
    for name,group in groups:
        count = group[var].count()
        dfAll['cnt_'+var].ix[group.index] = count
dfAll[added].to_csv('../../Cache/Feat_cnt_col_nonmissing.csv',index=False)

added = ['ID']
for tool,pro in procedure.items():
    if tool[-3:] in ['261','400','420']:
        continue
    groups = dfAll.groupby(['TOOL_'+tool[-3:]])
    for var in pro:
        if not var in dfAll.columns:
            continue
        dfAll['cnt_'+tool+'_'+var] = 0
        added.append('cnt_'+tool+'_'+var)
        for name,group in groups:
            grps = group.groupby([var])
            for name2,grp in grps:
                dfAll['cnt_'+tool+'_'+var].ix[grp.index] = float(len(grp))/float(len(group))
dfAll[added].to_csv('../../Cache/Feat_cnt_tool_nonmissing.csv',index=False)

In [ ]:
#percentile 计算某一个值在全集中的位置（百分制）
dfAll = dfAll.replace({-99999999:np.nan})
added = ['ID']
for var in raw_predictors:
    if not var in dfAll.columns:
        continue
    if 'TOOL' not in var:
        dfAll['pcent_'+var] = dfAll[var].rank(method='max')/float(len(dfAll))
        added.append('pcent_'+var)
dfAll[added].to_csv('../../Cache/Feat_pcent_col_nonmissing.csv',index=False)
t = 0
added = ['ID']
for tool,pro in procedure.items():
    if tool[-3:] in ['261','400','420']:
        continue
    groups = dfAll.groupby(['TOOL_'+tool[-3:]]) 
    for var in pro:
        if not var in dfAll.columns:
            continue
        dfAll['pcent_'+tool+'_'+var] = 0
        added.append('pcent_'+tool+'_'+var)
        for name,group in groups:
            dfAll['pcent_'+tool+'_'+var].ix[group.index] = group[var].rank(method='max')/float(len(group))
            
dfAll[added].to_csv('../../Cache/Feat_pcent_tool_nonmissing.csv',index=False) 

In [ ]:
len(raw_predictors)

In [6]:
##用unique值的交集做组合
minus_var = ['ID']
criteria = 0.7
for i in range(len(raw_predictors)-1):
    var1_unique = dfAll[raw_predictors[i]].unique().tolist()
    for j in range(i+1,len(raw_predictors)):
        var2_unique = dfAll[raw_predictors[j]].unique().tolist()
        combine_list = set(var1_unique+var2_unique)
        if float(len(var1_unique))/len(combine_list)>=criteria and float(len(var2_unique))/len(combine_list)>=criteria:
            tmp_var = raw_predictors[i]+'-'+raw_predictors[j]
            dfAll[tmp_var] = dfAll[raw_predictors[i]]-dfAll[raw_predictors[j]]
            minus_var.append(tmp_var)
            print(tmp_var)

210X14-210X16
210X14-210X32
210X19-220X481
210X19-220X486
210X19-220X488
210X19-220X489
210X19-311X43
210X19-330X906
210X19-330X1053
210X19-360X1002
210X19-400X56
210X19-750X1002
210X19-750X1164
210X20-210X23
210X25-210X128
210X25-210X200
210X25-210X202
210X25-210X211
210X26-210X129
210X26-210X201
210X26-210X203
210X26-210X212
210X27-210X57
210X28-210X58
210X28-210X65
210X28-210X66
210X28-210X124
210X28-210X125
210X28-210X138
210X28-210X139
210X29-210X98
210X29-210X99
210X37-210X80
210X37-220X535
210X37-310X38
210X37-261X318
210X37-261X319
210X37-261X322
210X37-261X323
210X37-261X688
210X37-261X694
210X37-261X695
210X37-312X307
210X37-312X308
210X37-312X310
210X37-312X311
210X37-312X421
210X37-312X430
210X37-340X97
210X37-340X105
210X37-344X210
210X37-750X712
210X40-210X42
210X41-210X43
210X44-210X46
210X45-210X47
210X53-210X55
210X53-210X116
210X53-210X122
210X54-210X56
210X54-210X117
210X54-210X123
210X55-210X116
210X55-210X122
210X56-210X117
210X56-210X123
210X57-210X58
210X57-210X1

220X394-220X395
220X403-312X111
220X404-261X239
220X404-261X425
220X404-261X455
220X404-261X611
220X404-261X641
220X404-330X2
220X417-520X389
220X424-420X192
220X424-420X193
220X424-420X195
220X424-420X196
220X429-330X1009
220X429-330X1016
220X433-311X17
220X433-311X20
220X433-312X474
220X433-312X477
220X433-312X486
220X433-344X246
220X450-312X197
220X471-220X476
220X471-220X480
220X476-220X480
220X477-220X478
220X477-220X539
220X477-261X321
220X477-261X689
220X477-261X693
220X477-312X331
220X477-312X332
220X477-312X334
220X477-312X335
220X477-340X57
220X477-340X99
220X477-344X198
220X477-344X199
220X477-344X201
220X477-344X202
220X477-360X712
220X478-220X483
220X478-220X487
220X478-220X539
220X478-261X689
220X478-312X331
220X478-312X332
220X478-312X334
220X478-312X335
220X478-340X99
220X478-360X712
220X481-220X486
220X481-220X488
220X481-220X489
220X481-311X43
220X481-330X906
220X481-330X1053
220X481-360X1002
220X481-400X56
220X481-750X1002
220X481-750X1164
220X483-220X487
220X483-261

311X86-311X87
311X86-311X90
311X86-311X91
311X86-311X164
311X86-311X165
311X86-311X168
311X86-311X170
311X86-311X171
311X86-311X173
311X86-311X174
311X86-311X175
311X86-311X178
311X86-311X179
311X86-261X351
311X86-261X354
311X86-261X356
311X86-261X357
311X86-261X359
311X86-261X360
311X86-261X361
311X86-261X364
311X86-261X365
311X86-261X536
311X86-261X537
311X86-261X540
311X86-261X542
311X86-261X543
311X86-261X545
311X86-261X546
311X86-261X547
311X86-261X550
311X86-261X551
311X86-261X722
311X86-261X723
311X86-261X726
311X86-261X728
311X86-261X729
311X86-261X731
311X86-261X732
311X86-261X733
311X86-261X736
311X86-261X737
311X87-311X90
311X87-311X91
311X87-311X164
311X87-311X165
311X87-311X168
311X87-311X170
311X87-311X171
311X87-311X173
311X87-311X174
311X87-311X175
311X87-311X178
311X87-311X179
311X87-261X351
311X87-261X354
311X87-261X356
311X87-261X357
311X87-261X359
311X87-261X360
311X87-261X361
311X87-261X364
311X87-261X365
311X87-261X536
311X87-261X537
311X87-261X540
311X87-261X542


311X179-261X351
311X179-261X354
311X179-261X356
311X179-261X357
311X179-261X359
311X179-261X360
311X179-261X361
311X179-261X364
311X179-261X365
311X179-261X536
311X179-261X537
311X179-261X540
311X179-261X542
311X179-261X543
311X179-261X545
311X179-261X546
311X179-261X547
311X179-261X550
311X179-261X551
311X179-261X722
311X179-261X723
311X179-261X726
311X179-261X728
311X179-261X729
311X179-261X731
311X179-261X732
311X179-261X733
311X179-261X736
311X179-261X737
311X182-261X554
311X182-261X740
311X183-311X184
311X183-311X185
311X183-261X369
311X183-261X370
311X183-261X371
311X183-261X555
311X183-261X556
311X183-261X557
311X183-261X741
311X183-261X742
311X183-344X7
311X184-311X185
311X184-261X369
311X184-261X370
311X184-261X371
311X184-261X555
311X184-261X556
311X184-261X557
311X184-261X741
311X184-261X742
311X185-261X369
311X185-261X371
311X185-261X555
311X185-261X557
311X185-261X741
311X185-261X743
311X191-311X192
311X191-311X193
311X191-261X377
311X191-261X378
311X192-311X193
311X192-26

261X244-261X254
261X244-261X264
261X244-261X274
261X244-261X284
261X244-261X421
261X244-261X430
261X244-261X440
261X244-261X450
261X244-261X460
261X244-261X481
261X244-261X616
261X244-261X626
261X244-261X636
261X244-261X646
261X244-261X667
261X245-261X255
261X245-261X265
261X245-261X275
261X245-261X431
261X245-261X441
261X245-261X451
261X245-261X461
261X245-261X617
261X245-261X627
261X245-261X637
261X245-261X647
261X246-261X256
261X246-261X266
261X246-261X276
261X246-261X284
261X246-261X286
261X246-261X294
261X246-261X420
261X246-261X422
261X246-261X432
261X246-261X442
261X246-261X452
261X246-261X462
261X246-261X470
261X246-261X472
261X246-261X480
261X246-261X606
261X246-261X608
261X246-261X618
261X246-261X628
261X246-261X638
261X246-261X648
261X246-261X656
261X246-261X658
261X246-261X666
261X247-261X257
261X247-261X268
261X247-261X413
261X247-261X424
261X247-261X433
261X247-261X443
261X247-261X454
261X247-261X463
261X247-261X599
261X247-261X609
261X247-261X610
261X247-261X619
261X247-

...

340X173-340X175
340X173-340X177
340X175-340X177
340X181-344X347
340X182-344X349
340X185-340X186
340X185-340X187
340X185-344X54
340X185-344X55
340X185-344X57
340X185-344X58
340X185-344X60
340X185-344X61
340X185-344X355
340X185-344X356
340X185-344X358
340X185-344X359
340X185-344X361
340X185-344X362
340X186-340X187
340X186-344X54
340X186-344X55
340X186-344X57
340X186-344X58
340X186-344X60
340X186-344X61
340X186-344X355
340X186-344X356
340X186-344X358
340X186-344X359
340X186-344X361
340X186-344X362
340X187-344X54
340X187-344X55
340X187-344X57
340X187-344X58
340X187-344X60
340X187-344X61
340X187-344X355
340X187-344X356
340X187-344X358
340X187-344X359
340X187-344X361
340X187-344X362
340X188-340X189
340X188-340X190
340X188-344X364
340X188-344X365
340X188-344X367
340X188-344X368
340X188-344X370
340X188-344X371
340X189-340X190
340X189-344X364
340X189-344X365
340X189-344X367
340X189-344X368
340X189-344X370
340X189-344X371
340X190-344X364
340X190-344X365
340X190-344X367
340X190-344X368
340X190-34

400X134-400X136
400X134-400X138
400X134-400X139
400X134-400X140
400X134-400X141
400X134-400X142
400X134-420X114
400X134-420X115
400X134-420X116
400X134-420X117
400X134-420X118
400X134-420X120
400X134-420X123
400X134-420X124
400X134-420X132
400X134-420X136
400X134-420X150
400X134-420X151
400X134-420X152
400X134-420X153
400X134-420X154
400X134-520X305
400X134-520X306
400X134-520X307
400X134-520X308
400X134-520X309
400X134-520X323
400X134-520X324
400X134-520X325
400X134-520X326
400X134-520X327
400X134-520X329
400X134-520X330
400X134-520X331
400X134-520X333
400X135-400X151
400X135-400X152
400X135-400X154
400X135-400X156
400X135-400X157
400X135-400X158
400X135-400X159
400X135-400X160
400X135-420X135
400X135-420X156
400X135-420X157
400X135-420X159
400X135-420X160
400X136-400X138
400X136-400X139
400X136-400X140
400X136-400X141
400X136-400X142
400X136-420X114
400X136-420X115
400X136-420X116
400X136-420X117
400X136-420X118
400X136-420X120
400X136-420X123
400X136-420X124
400X136-420X132
400X136-

420X120-420X123
420X120-420X124
420X120-420X132
420X120-420X136
420X120-420X150
420X120-420X151
420X120-420X152
420X120-420X153
420X120-420X154
420X120-520X305
420X120-520X306
420X120-520X307
420X120-520X308
420X120-520X309
420X120-520X323
420X120-520X324
420X120-520X325
420X120-520X326
420X120-520X327
420X120-520X329
420X120-520X330
420X120-520X331
420X120-520X333
420X121-420X122
420X121-420X134
420X121-520X281
420X121-520X282
420X121-520X283
420X121-520X284
420X121-520X285
420X122-420X134
420X122-520X281
420X122-520X282
420X122-520X283
420X122-520X284
420X122-520X285
420X123-420X124
420X123-420X132
420X123-420X136
420X123-420X150
420X123-420X151
420X123-420X152
420X123-420X153
420X123-420X154
420X123-520X305
420X123-520X306
420X123-520X307
420X123-520X308
420X123-520X309
420X123-520X323
420X123-520X324
420X123-520X325
420X123-520X326
420X123-520X327
420X123-520X329
420X123-520X330
420X123-520X331
420X123-520X333
420X124-420X132
420X124-420X136
420X124-420X150
420X124-420X151
420X124-

520X28-520X108
520X31-520X32
520X31-520X37
520X31-520X46
520X31-520X47
520X31-520X63
520X32-520X37
520X32-520X46
520X32-520X47
520X32-520X61
520X32-520X63
520X32-520X78
520X36-520X67
520X36-520X72
520X36-520X103
520X36-520X108
520X37-520X46
520X37-520X47
520X37-520X97
520X38-520X66
520X38-520X67
520X38-520X97
520X41-520X42
520X41-520X43
520X41-520X51
520X41-520X88
520X42-520X43
520X42-520X51
520X42-520X52
520X42-520X86
520X42-520X88
520X43-520X51
520X43-520X53
520X43-520X88
520X46-520X47
520X47-520X61
520X47-520X63
520X47-520X78
520X51-520X88
520X52-520X68
520X52-520X86
520X52-520X87
520X52-520X88
520X56-520X57
520X56-520X58
520X56-520X73
520X56-520X83
520X57-520X58
520X57-520X71
520X57-520X73
520X57-520X81
520X58-520X66
520X58-520X73
520X58-520X83
520X58-520X96
520X61-520X62
520X61-520X63
520X61-520X76
520X61-520X78
520X62-520X76
520X62-520X77
520X62-520X78
520X62-520X102
520X62-520X113
520X63-520X78
520X66-520X67
520X66-520X68
520X66-520X87
520X66-520X96
520X66-520X97
520X67-520X68
5

750X435-750X454
750X435-750X465
750X435-750X476
750X435-750X487
750X435-750X498
750X435-750X509
750X435-750X520
750X435-750X531
750X435-750X583
750X435-750X594
750X435-750X604
750X435-750X628
750X435-750X636
750X435-750X676
750X446-750X465
750X446-750X476
750X446-750X487
750X446-750X498
750X446-750X520
750X446-750X531
750X446-750X572
750X446-750X583
750X446-750X636
750X454-750X465
750X454-750X476
750X454-750X487
750X454-750X498
750X454-750X509
750X454-750X520
750X454-750X531
750X454-750X572
750X454-750X583
750X454-750X594
750X454-750X604
750X454-750X612
750X454-750X620
750X454-750X628
750X454-750X636
750X454-750X644
750X454-750X676
750X465-750X476
750X465-750X487
750X465-750X498
750X465-750X520
750X465-750X531
750X465-750X553
750X465-750X564
750X465-750X572
750X465-750X583
750X465-750X604
750X465-750X628
750X465-750X636
750X465-750X668
750X465-750X676
750X476-750X487
750X476-750X498
750X476-750X520
750X476-750X531
750X476-750X572
750X476-750X583
750X476-750X604
750X476-750X620
750X476-

In [7]:
len(minus_var)

8594

In [9]:
dfAll[minus_var].to_csv('../../Cache/All_minus.csv',index=False)

In [24]:
dffAll_minus = pd.read_csv('../../../All_minus_single_col.csv')

In [11]:
cnt_list =['ID']
for var in dfAll:
    tmp_var = 'cnt_'+var
    if tmp_var in dffAll_minus.columns:
        cnt_list.append(tmp_var)
        cnt_list.append('pcent_'+var)
print(len(cnt_list))

5007


In [22]:
dffAll_minus[added].to_csv('../../Cache/All_minus_single_col.csv',index=False)

In [9]:
dffAll_minus = pd.read_csv('../../Cache/All_minus.csv')

In [ ]:
minus_var = ['ID']
for i in range(len(raw_predictors)-1):
    for j in range(i+1,len(raw_predictors)):
        if (dfAll[raw_predictors[i]].max()-dfAll[raw_predictors[j]].max())*(dfAll[raw_predictors[i]].min()-dfAll[raw_predictors[j]].min())<0:
            if float(min(dfAll[raw_predictors[i]].max()-dfAll[raw_predictors[i]].min(),dfAll[raw_predictors[j]].max()-dfAll[raw_predictors[j]].min()))/max(dfAll[raw_predictors[i]].max()-dfAll[raw_predictors[i]].min(),dfAll[raw_predictors[j]].max()-dfAll[raw_predictors[j]].min())>=0.6:
                tmp_var = raw_predictors[i]+'-'+raw_predictors[j]
                dfAll[tmp_var] = dfAll[raw_predictors[i]]-dfAll[raw_predictors[j]]
                minus_var.append(tmp_var)
                print(tmp_var)
        elif (dfAll[raw_predictors[i]].max()-dfAll[raw_predictors[j]].min())*(dfAll[raw_predictors[j]].max()-dfAll[raw_predictors[i]].min())>0:
            if float(min(abs(dfAll[raw_predictors[i]].max()-dfAll[raw_predictors[j]].min()),abs(dfAll[raw_predictors[j]].max()-dfAll[raw_predictors[i]].min())))/max(abs(dfAll[raw_predictors[i]].max()-dfAll[raw_predictors[j]].min()),abs(dfAll[raw_predictors[j]].max()-dfAll[raw_predictors[i]].min()))>=0.6:
                tmp_var = raw_predictors[i]+'-'+raw_predictors[j]
                dfAll[tmp_var] = dfAll[raw_predictors[i]]-dfAll[raw_predictors[j]]
                minus_var.append(tmp_var)
                print(tmp_var)

In [12]:
dffAll_minus = dfAll[minus_var]
#dffAll_minus.to_csv('../../../All_minus.csv',index=False)

In [3]:
dffAll_minus = pd.read_csv('../../../All_minus.csv')

In [4]:
minus_var = dffAll_minus.columns.tolist()

In [10]:
dffAll_minus.shape

(721, 8594)

In [11]:
added = ['ID']
t=0
for var in minus_var[1:]:
    if dffAll_minus[var].nunique()>10:
        continue
    t+=1
    print(t)
    dffAll_minus['cnt_'+var] = 0
    added.append('cnt_'+var)
    if 'cnt_'+var in dffAll_minus.columns:
        continue
    groups = dffAll_minus.groupby([var])
    for name,group in groups:
        count = group[var].count()
        dffAll_minus['cnt_'+var].ix[group.index] = count
for var in minus_var[1:]:
    if 'TOOL' not in var:
        dffAll_minus['pcent_'+var] = dffAll_minus[var].rank(method='max')/float(len(dffAll_minus))
        added.append('pcent_'+var)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060
2061
2062
2063
2064
2065
2066
2067
2068
2069
2070
2071
2072
2073
2074
2075
2076
2077
2078
2079
2080
2081
2082
2083
2084
2085
2086
2087
2088
2089
2090
2091
2092
2093
2094
2095
2096
2097
2098
2099
2100
2101
2102
2103
2104
2105
2106
2107
2108
2109
2110
2111
2112
2113
2114
2115
2116
2117
2118
2119
2120
2121
2122
2123
2124
2125
2126
2127
2128
2129
2130
2131
2132
2133
2134
2135
2136
2137
2138


3672
3673
3674
3675
3676
3677
3678
3679
3680
3681
3682
3683
3684
3685
3686
3687
3688
3689
3690
3691
3692
3693
3694
3695
3696
3697
3698
3699
3700
3701
3702
3703
3704
3705
3706
3707
3708
3709
3710
3711
3712
3713
3714
3715
3716
3717
3718
3719
3720
3721
3722
3723
3724
3725
3726
3727
3728
3729
3730
3731
3732
3733
3734
3735
3736
3737
3738
3739
3740
3741
3742
3743
3744
3745
3746
3747
3748
3749
3750
3751
3752
3753
3754
3755
3756
3757
3758
3759
3760
3761
3762
3763
3764
3765
3766
3767
3768
3769
3770
3771
3772
3773
3774
3775
3776
3777
3778
3779
3780
3781
3782
3783
3784
3785
3786
3787
3788
3789
3790
3791
3792
3793
3794
3795
3796
3797
3798
3799
3800
3801
3802
3803
3804
3805
3806
3807
3808
3809
3810
3811
3812
3813
3814
3815
3816
3817
3818
3819
3820
3821
3822
3823
3824
3825
3826
3827
3828
3829
3830
3831
3832
3833
3834
3835
3836
3837
3838
3839
3840
3841
3842
3843
3844
3845
3846
3847
3848
3849
3850
3851
3852
3853
3854
3855
3856
3857
3858
3859
3860
3861
3862
3863
3864
3865
3866
3867
3868
3869
3870
3871


In [12]:
len(added)

12527

In [7]:
for var in minus_var[1:]:
    if 'TOOL' not in var:
        dffAll_minus['pcent_'+var] = dffAll_minus[var].rank(method='max')/float(len(dffAll_minus))
        added.append('pcent_'+var)

In [10]:
dffAll_minus[added].to_csv('../../Cache/All_minus_single_col.csv',index=False)

In [19]:
dfAll.columns

Index(['ID', '210X1', '210X2', '210X3', '210X4', '210X5', '210X6', '210X7',
       '210X8', '210X9',
       ...
       '520X330-520X333', '520X331-520X333', '520X345-520X353',
       '520X387-520X390', '520X387-520X391', '520X390-520X391',
       '520X403-520X407', '750X27-750X1071', '750X1002-750X1164',
       '750X1049-750X1071'],
      dtype='object', length=7828)

In [27]:
tool = ['ID']
for var in dfAll.columns:
    if 'TOOL' in var:
        tool.append(var)
dffAll_minus = dffAll_minus.merge(dfAll[tool],'inner','ID')
dffAll_minus.shape

(721, 8607)

In [28]:
minus_tool_dict ={}

for var1 in tool[1:]+['261','400','420']:
    var1_num = var1[-3:]
    minus_tool_dict[var1_num] ={}
    tmp = [i for i in dffAll_minus.columns if i[0:3]==var1_num]
    for var2 in tool[1:]+['261','400','420']:
        var2_num = var2[-3:]
        minus_tool_dict[var1_num][var2_num] =[i for i in tmp if i[i.find('-')+1:i.find('-')+4]==var2_num]
minus_tool_dict
        

{'210': {'210': ['210X14-210X16',
   '210X14-210X32',
   '210X20-210X23',
   '210X25-210X128',
   '210X25-210X200',
   '210X25-210X202',
   '210X25-210X211',
   '210X26-210X129',
   '210X26-210X201',
   '210X26-210X203',
   '210X26-210X212',
   '210X27-210X57',
   '210X28-210X58',
   '210X28-210X65',
   '210X28-210X66',
   '210X28-210X124',
   '210X28-210X125',
   '210X28-210X138',
   '210X28-210X139',
   '210X29-210X98',
   '210X29-210X99',
   '210X37-210X80',
   '210X40-210X42',
   '210X41-210X43',
   '210X44-210X46',
   '210X45-210X47',
   '210X53-210X55',
   '210X53-210X116',
   '210X53-210X122',
   '210X54-210X56',
   '210X54-210X117',
   '210X54-210X123',
   '210X55-210X116',
   '210X55-210X122',
   '210X56-210X117',
   '210X56-210X123',
   '210X57-210X58',
   '210X57-210X125',
   '210X57-210X138',
   '210X58-210X65',
   '210X58-210X66',
   '210X58-210X124',
   '210X58-210X125',
   '210X58-210X138',
   '210X58-210X139',
   '210X60-210X67',
   '210X60-210X68',
   '210X60-210X75',


In [ ]:
added = ['ID']
for key1,value1 in minus_tool_dict.items():
    for key2,value2 in value1.items():
        if key1==key2:
            if key1 in ['261','400','420']:
                continue
            else:
                groups = dffAll_minus.groupby(['TOOL_'+key1])
        else:
            if key1 in ['261','400','420'] and key2 in ['261','400','420']:
                continue
            if key1 in ['261','400','420']:
                groups = dffAll_minus.groupby(['TOOL_'+key2])
            if key2 in ['261','400','420']:
                groups = dffAll_minus.groupby(['TOOL_'+key1])
            else:
                groups = dffAll_minus.groupby(['TOOL_'+key1,'TOOL_'+key2])
        print(key1,key2)
        for var in value2:
            dffAll_minus['cnt_tool_'+var] = 0
            dffAll_minus['pcent_tool_'+var] = 0
            added.append('cnt_tool_'+var)
            added.append('pcent_tool_'+var)
            for name,group in groups:
                dffAll_minus['pcent_tool_'+var].ix[group.index] = group[var].rank(method='max')/float(len(group))
                grps = group.groupby([var])
                for name2,grp in grps:
                    dffAll_minus['cnt_tool_'+var].ix[grp.index] = float(len(grp))/float(len(group))

    
'''for tool,pro in procedure.items():
    if tool[-3:] in ['261','400','420']:
        continue
    groups = dfAll.groupby(['TOOL_'+tool[-3:]])
    for var in pro:
        if not var in dfAll.columns:
            continue
        dfAll['cnt_'+tool+'_'+var] = 0
        added.append('cnt_'+tool+'_'+var)
        for name,group in groups:
            grps = group.groupby([var])
            for name2,grp in grps:
                dfAll['cnt_'+tool+'_'+var].ix[grp.index] = float(len(grp))/float(len(group))
                
                
                
                
for tool,pro in procedure.items():
    if tool[-3:] in ['261','400','420']:
        continue
    groups = dfAll.groupby(['TOOL_'+tool[-3:]]) 
    for var in pro:
        if not var in dfAll.columns:
            continue
        dfAll['pcent_'+tool+'_'+var] = 0
        added.append('pcent_'+tool+'_'+var)
        for name,group in groups:
            dfAll['pcent_'+tool+'_'+var].ix[group.index] = group[var].rank(method='max')/float(len(group))'''

210 210


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:28: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


210 220
210 300
210 310
210 311
210 312
210 330
210 340
210 344
210 360
210 440
210 520
210 750
210 261
210 400
210 420
220 210
220 220
220 300
220 310
220 311
220 312
220 330
220 340
220 344
220 360
220 440
220 520
220 750
220 261
220 400
220 420
300 210
300 220
300 300
300 310
300 311
300 312
300 330
300 340
300 344
300 360
300 440
300 520
300 750
300 261
300 400
300 420
310 210
310 220
310 300
310 310
310 311
310 312
310 330
310 340
310 344
310 360
310 440
310 520
310 750
310 261
310 400
310 420
311 210
311 220
311 300
311 310
311 311
311 312
311 330
311 340
311 344
311 360
311 440
311 520
311 750
311 261
311 400
311 420
312 210
312 220
312 300
312 310
312 311
312 312


In [30]:
dffAll_minus[added].to_csv('../../Cache/All_minus_byother_col.csv',index=False)

In [ ]:
#Pivot 创建多组category variable 的两两交叉表,感觉只能用来计算分类型变量,好像现在直接可以用pandas解决
def get_pivottable(X_train, X_test, use='all',Feat_list):
    """
    Returns a list of dictionaries, one per feature in the
    basic data, containing cross-tabulated counts
    for each column and each value of the feature.
    """
    dictionaries = []
    if use == 'all':
        X = np.vstack((X_train, X_test))
        filename = "pivottable"
    elif use == 'train':
        X = X_train
        filename = "pivottable_train"
    else:
        X = X_test
        filename = "pivottable_test"
    for i in range(len(Feat_list)):
        dictionaries.append({'total': 0})
    try:
        with open("cache/%s.pkl" % filename, 'rb') as f:
            logger.debug("loading cross-tabulated data from cache")
            dictionaries = pickle.load(f)
    except IOError:
        logger.debug("no cache found, cross-tabulating data")
        for i, row in enumerate(X):
            for j in range(len(Feat_list)):
                dictionaries[j]['total'] += 1
                if row[j] not in dictionaries[j]:
                    dictionaries[j][row[j]] = {'total': 1}
                    for k, key in enumerate(Feat_list):
                        dictionaries[j][row[j]][key] = {row[k]: 1}
                else:
                    dictionaries[j][row[j]]['total'] += 1
                    for k, key in enumerate(Feat_list):
                        if row[k] not in dictionaries[j][row[j]][key]:
                            dictionaries[j][row[j]][key][row[k]] = 1
                        else:
                            dictionaries[j][row[j]][key][row[k]] += 1
        with open("cache/%s.pkl" % filename, 'wb') as f:
            pickle.dump(dictionaries, f, pickle.HIGHEST_PROTOCOL)

    return dictionaries

def creat_cate_corr(X_train, X_test, Feat_list=[var if 'TOOL' in var for var in raw_predictor]):
    features_train = []
    features_test = []
    dictionaries = get_pivottable(X_train[Feat_list], X_test[Feat_list],Feat_list=Feat_list)
    #dictionaries_train = get_pivottable(X_train[Feat_list], X_test[Feat_list], use='train',Feat_list=Feat_list)
    #dictionaries_test = get_pivottable(X_test[Feat_list], X_test[Feat_list], use='test',Feat_list=Feat_list)

In [ ]:
dfAll = pd.read_csv('../../Data/All_Data_dedup.csv')
feat_needed = dfAll.columns.tolist()[14:]
'''gap_list = []
for i in range(len(feat_needed)-1):
    for j in range(i+1,len(feat_needed)):
        if dfAll[feat_needed[i]].quantile(0.5) - dfAll[feat_needed[j]].quantile(0.5)<=1 and dfAll[feat_needed[i]].quantile(0.5) - dfAll[feat_needed[j]].quantile(0.5)>=-1:
            tmp_var = feat_needed[i]+'-'+feat_needed[j]
            dfAll[tmp_var] = dfAll[feat_needed[i]]- dfAll[feat_needed[j]]
            gap_list.append(tmp_var)
            print(tmp_var)'''

In [ ]:
feat_needed2 =[]
for var in feat_needed:
    if dfAll[var].nunique()>15:
        feat_needed2.append(var)
gap_list = []
for i in range(len(feat_needed2)-1):
    for j in range(i+1,len(feat_needed2)):
        if dfAll[feat_needed2[i]].quantile(0.5) - dfAll[feat_needed2[j]].quantile(0.5)<=2 and dfAll[feat_needed2[i]].quantile(0.5) - dfAll[feat_needed2[j]].quantile(0.5)>=-2:
            tmp_var = feat_needed2[i]+'-'+feat_needed2[j]
            dfAll[tmp_var] = dfAll[feat_needed2[i]]- dfAll[feat_needed2[j]]
            gap_list.append(tmp_var)
            print(tmp_var)

In [ ]:
dfAll = dfAll.set_index(dfAll['ID'])
dfTrain_tmp = dfAll.loc[dfTrain['ID']]
dfPred_tmp = dfAll.loc[dfPred['ID']]

In [ ]:
dfPred = dfPred_tmp.copy(deep=True)
dfTrain = dfTrain_tmp.copy(deep=True)

In [ ]:
Feat_file = ['Feat_cnt_col','Feat_cnt_tool','Feat_pcent_col','Feat_pcent_tool']
ex_feat = []
for filename in Feat_file:
    tmp = pd.read_csv('../../Cache/'+filename+'.csv')
    tmp.set_index(tmp['ID'],inplace=True,drop=True)
    tmp_feat_list = tmp.columns.tolist()
    if 'ID' in tmp_feat_list:
        tmp_feat_list.remove('ID')
    ex_feat += tmp_feat_list
    dfTrain = pd.merge(dfTrain,tmp,'left','ID')
    dfPred = pd.merge(dfPred,tmp,'left','ID')

In [ ]:
dfAll2 = pd.read_csv('../../Data/All_Data_dedup.csv',nrows=2)
raw_predictors = dfAll2.columns[1:]

In [ ]:
dfTrain.columns

In [ ]:
f = open('../../Cache/var_change.txt','r')
a = f.read()
var_change = eval(a)
f.close()
predictors = dfTrain.columns.tolist()[1:]+ex_feat
feat_cnt = 0
for key,value in var_change.items():
    feat_cnt+=1
    if feat_cnt>1:
        continue
    for var in var_change[key]['constant']:
        try:
             predictors.remove(var)
        except:
            continue
    for var in var_change[key]['category']:
        try:
            predictors.remove(var)
        except:
            continue
        tmpTrain = pd.get_dummies(dfTrain[var],prefix=var,dummy_na=True)
        tmpPred = pd.get_dummies(dfPred[var],prefix=var,dummy_na=True)
        predictors = predictors + tmpTrain.columns.tolist()
        dfTrain = pd.concat([dfTrain,tmpTrain],axis=1)
        dfPred = pd.concat([dfPred,tmpPred],axis=1)
for var in predictors:
    if var not in dfPred.columns:
        dfPred[var] = 0

In [ ]:
def xgb_kfold(dfTrain,dfPred,predictors,n_splits=5, params = {'max_depth':3, 'eta':0.01, 'silent':0,'objective':'reg:linear','lambda':1,'subsample':0.8,
                         'colsample_bytree':0.4}):  
    kf = KFold(n_splits=n_splits,shuffle=True)
    dpred = xgb.DMatrix(dfPred[predictors].values,label=[0]*len(dfPred),missing=np.nan)
    imp = pd.DataFrame({'variable':predictors,'lk':['f'+str(i) for i in range(len(predictors))]})
    round=0
    for train_index, test_index in kf.split(dfTrain):
        round+=1
        train_X = dfTrain.loc[train_index,predictors]
        test_X = dfTrain.loc[test_index,predictors]
        train_Y = dfTrain.loc[train_index,'Y']
        test_Y = dfTrain.loc[test_index,'Y']

        dtrain = xgb.DMatrix(train_X.values, label=train_Y.values, missing = np.nan)
        dtest = xgb.DMatrix(test_X.values, label=test_Y.values, missing = np.nan)
        param = params 
        evallist  = [(dtrain,'train'),(dtest,'eval')]  
        num_round = 5000
        evals_dict = {}
        model = xgb.train(param,dtrain,num_round, evallist,early_stopping_rounds=50,evals_result=evals_dict,verbose_eval =1000)
        performance_df = pd.DataFrame(evals_dict['eval'])
        bst_tree = len(performance_df)-51
        pred_test = model.predict(dtest,ntree_limit =bst_tree)

        tmp_imp = pd.DataFrame(model.get_score(importance_type='gain'),index=['imp_fold%d'%round]).T
        tmp_imp['lk'] = tmp_imp.index
        imp = imp.merge(tmp_imp,'left','lk').fillna(0)


        pred_score = model.predict(dpred,ntree_limit =bst_tree)
        if round==1:
            test_result = pd.DataFrame({'ID':dfTrain.loc[test_index,'ID'].values,'score':pred_test,'target':test_Y})
            result = pd.DataFrame({'ID':dfPred['ID'],'Score_%d'%round:pred_score})
        else:
            test_result = pd.concat([test_result,pd.DataFrame({'ID':dfTrain.loc[test_index,'ID'].values,'score':pred_test,'target':test_Y})],axis=0)
            result = result.merge(pd.DataFrame({'ID':dfPred['ID'],'Score_%d'%round:pred_score}),'inner','ID')
    #print("Test MSE:",metrics.mean_squared_error(test_result['target'], test_result['score']))
    return test_result,result,imp

In [ ]:
dfTrain2 = pd.read_csv(config.original_train_data_path,usecols=['ID','Y'])
dfTrain = dfTrain.merge(dfTrain2,'inner','ID')
dfTrain.shape

In [ ]:
test_result,result,imp = xgb_kfold(dfTrain,dfPred,predictors)

In [ ]:
result['score']=result[['Score_1','Score_2','Score_3','Score_4','Score_5']].mean(axis=1)
submit = result[['ID','score']]
today = datetime.date.today().strftime('%Y-%m-%d')
submit.to_csv('../../Submission/submit_%s.csv'%today,header=False,index=False)
imp.to_csv('../../Profiling/imp_test_v3_%s.csv'%today,index=False)
test_result('../../Profiling/test_score_%s.csv'%today,index=False)